# Best Version of Federated Learning across Nodes Training a CNN model

New script to run a CNN-based federated system, where we can vary the inner and outer learning rates

In [1]:
#Imports
import os    
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
from sklearn import preprocessing
from sklearn import metrics
from keras.datasets import mnist
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import random
import csv
import timeit
from timeit import default_timer as timer

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer','dog', 'frog', 'horse', 'ship', 'truck']
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

Functions to deal with splitting of data or creating models. Basically non-interesting functions

In [5]:
def splittingData(x_train,y_train,x_test,y_test,noClients):
    trainLen = len(x_train)
    clientSize = trainLen/noClients
    
    x_train_clients = []
    y_train_clients = []
    x_test_clients = []
    y_test_clients = []
    
    x_train_splits = np.array_split(x_train,noClients)
    y_train_splits = np.array_split(y_train,noClients)
    x_test_splits = np.array_split(x_test,noClients)
    y_test_splits = np.array_split(y_test,noClients)
    
    for i in range(noClients):
        x_train_clients.append(x_train_splits[i])
        y_train_clients.append(y_train_splits[i])
        x_test_clients.append(x_test_splits[i])
        y_test_clients.append(y_test_splits[i])
    
    return x_train_clients, y_train_clients, x_test_clients, y_test_clients

def createModel(iLr):
    #optimizer = tf.keras.optimizers.Adam(learning_rate=iLr)
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10))
    model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
    return model

def createZeroWeightCNN(iLr):
    layer1 = [np.zeros(shape = (3,3,3,32), dtype = 'float32'), np.zeros(shape = 32, dtype = 'float32')]
    layer2 = [np.zeros(shape = (3,3,32,64), dtype = 'float32'), np.zeros(shape = 64, dtype = 'float32')]
    layer3 = [np.zeros(shape = (10816,64), dtype = 'float32'), np.zeros(shape = (64), dtype = 'float32')]
    layer4 = [np.zeros(shape = (64,10), dtype = 'float32'), np.zeros(shape = (10), dtype = 'float32')]

    testModel = models.Sequential()
    testModel.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    testModel.add(layers.MaxPooling2D((2, 2)))
    testModel.add(layers.Conv2D(64, (3, 3), activation='relu'))
    testModel.add(layers.Flatten())
    testModel.add(layers.Dense(64, activation='relu'))
    testModel.add(layers.Dense(10))
    optimizer = tf.keras.optimizers.Adam(learning_rate=iLr)
    testModel.layers[0].set_weights(layer1)
    testModel.layers[2].set_weights(layer2)
    testModel.layers[4].set_weights(layer3)
    testModel.layers[5].set_weights(layer4)
    testModel.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
    return testModel



Federated Functions

In [6]:
def McMahanFedAvg(models_,fedModel,numClients,l):
    #setup an array to hold the weights and a null array to hold the summation of weights in the federated system
    layersNums = [0,2,4,5]    
    fedModelWeights = []
    mmSum = []
    for i in layersNums:
        fedModelWeights.append(fedModel.layers[i].weights)
        mmSum.append(fedModel.layers[i].weights)
    
    for i in range(len(layersNums)):
        for j in range(len(models_[0].layers[layersNums[i]].weights)):
            mmSum[i][j] = mmSum[i][j]*0
    
    #Perform the summation
    for n in range(0,numClients):
        count = 0
        for i in layersNums:
            for j in range(len(models_[0].layers[i].weights)):
                mmSum[count][j] = models_[n].layers[i].weights[j] - fedModelWeights[count][j]
            count += 1
    
    #find the new global models parameters
    count = 0
    for i in layersNums:
        for j in range(len(models_[0].layers[i].weights)):
            mmSum[count][j] = (l/numClients)*mmSum[count][j]
            fedModelWeights[count][j] = fedModelWeights[count][j] + mmSum[count][j]
        count += 1
    
    #Update the global model
    count = 0
    for i in layersNums:
        fedModel.layers[i].set_weights(fedModelWeights[count])
        count += 1
    return fedModel


def runIndivModelsFed(numClients, noEpochs, writer, oLr,iLr, v):
    #create local and global models
    row = []
    fedModel = createModel(iLr)
    models = []
    for i in range(numClients):
        model = createModel(iLr)
        models.append(model)
        
    #training the CNN
    for ep in range(0,noEpochs):
        start = timer()
        for i in range(numClients):
            models[i] = fedModel
            models[i].fit(x_train_clients[i], y_train_clients[i], epochs=1, verbose=v)
        fedModel = McMahanFedAvg(models,fedModel,numClients,oLr)
        #testing the CNN
        testModelsFed(ep,fedModel,numClients,writer,start,numClients,v,0)

    return fedModel, models

#Testing the model
def testModelsFed(ep,model,numClients,writer,start,NC,v,print_):
    fedAcc = 0
    row = [numClients, ep+1,0,0]
    for i in range (NC):
        row.append(0)
    for i in range(numClients):
        #tesing takes place here
        loss, acc = model.evaluate(x_test_clients[i],  y_test_clients[i], verbose=v)
        fedAcc += acc/numClients
        row[4+i] = acc
    print("Accuracy at epoch {0} is = {1}".format(ep,fedAcc))
    end = timer()
    row[2] = fedAcc
    row[3] = end-start
    #writing to .txt file
    if (print_ == 1):
        writer.writerow(row)

In [8]:
#Setting Seeds
os.environ['PYTHONHASHSEED']=str(2)
tf.random.set_seed(2)
np.random.seed(2)
random.seed(2)
#Variables to alter
nClients = 1
nEpochs = 10
outerLr = 0.5
innerLr = 0.001
verbose = 0
print_ = 0
#Opening a file to print into
f = open('./FCNNResults.csv', 'w', newline = '')
header = ['NumClients', 'NumEpochs','FTA','Time']
clients = []
for i in range(nClients):
    header.append("LAN{0}".format(i))
    clients.append(i+1)
writer = csv.writer(f)
if (print_ == 1):
    writer.writerow(header)

#Split into 'numClients' number of data sections
x_train_clients, y_train_clients, x_test_clients, y_test_clients = splittingData(train_images, train_labels, test_images, test_labels, nClients)

#Running the model
print("Running a federated system with {0} clients over {1} epochs".format(nClients,nEpochs))
fedModel, finalLocalModels = runIndivModelsFed(nClients, nEpochs, writer,outerLr,innerLr,verbose)

Running a federated system with 1 clients over 10 epochs
Accuracy at epoch 0 is = 0.5914999842643738


KeyboardInterrupt: 